In [1]:
from sshtunnel import SSHTunnelForwarder

# 0.) Setup up SSH tunnel
tunnel = SSHTunnelForwarder(
    ('207.148.5.16', 22),
    ssh_username='root',
    ssh_password='BaconTurkeyBrav0',
    remote_bind_address=('127.0.0.1', 3306),
    local_bind_address=('127.0.0.1', 3310)
)
tunnel.start()

In [2]:
import mysql.connector
import pandas as pd

# 1.) Connect to MySQL using tunnel
cnx = mysql.connector.connect(
    host='127.0.0.1',
    port=tunnel.local_bind_port,
    user='root',
    password='Synt@urNB7',
    database='SynD'
)

# 2.) Create cursor
cursor = cnx.cursor(dictionary=True, buffered=False)

# 3.) Query to fetch items
query = """
SELECT ItemNumber, iNote, CheckNote
FROM Item
"""
cursor.execute(query)

all_rows = []
for row in cursor:
    all_rows.append(row)

# 4.) Load into Pandas DataFrame for convenience
df_items = pd.DataFrame(all_rows)

cursor.close()
cnx.close()

In [3]:
tunnel.close()

In [4]:
df_items.head()

,ItemNumber,iNote,CheckNote
0,01000,,0.0
1,0705A,tech makes these. Parts in kit: CD4013N (Synt...,1.0
2,0705B,"Parts in kit: CD4013N (Syntaur #3322), 0.1 uF ...",0.0
3,10000,-- ALL ORDERS FOR THIS HAVE BEEN TEST ORDERS --,0.0
4,10001,,0.0


In [5]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# function for splitting sentences
def split_into_sentences(text):
    if not text:
        return []
    return sent_tokenize(text)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jivemachine/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# create labeled dataset
labeled_data = []
for idx, row in df_items.iterrows():
    note = row['iNote']
    sentences = split_into_sentences(note)
    for s in sentences:
        labeled_data.append({
            'ItemNumber': row['ItemNumber'],
            'sentence': s,
            'checkNoteRelevant': None # fill in manually with labeling tool
        })

df_labeled = pd.DataFrame(labeled_data)
df_labeled.to_csv("labeled_sentences.csv", index=False)